In [22]:
import pandas as pd
import csv
import scipy

import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2

In [23]:
def strip(s):
    return s.lstrip('"').rstrip('"')
def UTF16(s):
    ss=re.sub(r"\\u[0-9A-Fa-f]{4}",'',s)[:]
    return ss
def Leading(s):
    return s.lstrip()

In [24]:
train_raw = pd.read_csv('train-raw.tsv',sep = "\t", header = None,quoting=csv.QUOTE_NONE)
test_raw = pd.read_csv('test-raw.tsv',sep = "\t", header = None,quoting=csv.QUOTE_NONE) #dev raw
answer_raw = pd.read_csv('test-raw.tsv',sep = "\t", header = None,quoting=csv.QUOTE_NONE)

train_raw = train_raw.iloc[:5000,:]
test_raw =test_raw.iloc[:5000,:]

train_raw.columns = ['ID', 'Class','Twitter']
test_raw.columns = ['ID', 'Class','Twitter']
answer_raw.columns = ['id', 'Class','Twitter']

train_raw['Twitter']=train_raw['Twitter'].map(strip).map(UTF16).map(Leading)
test_raw['Twitter']=test_raw['Twitter'].map(strip).map(UTF16).map(Leading)
answer_raw['Twitter']=answer_raw['Twitter'].map(strip).map(UTF16).map(Leading)

train_text = train_raw['Twitter']
test_text = test_raw['Twitter']
answer_text = answer_raw['Twitter']

train_label=train_raw['Class']
test_label=test_raw['Class']
answer_label=answer_raw['Class']


In [25]:
vectoriser = CountVectorizer(encoding='utf-8',decode_error='strict' )
X_train = vectoriser.fit_transform(train_text) #bags of words
X_test = vectoriser.transform(test_text ) 
X_answer=vectoriser.transform(answer_text ) 
'''k=100
x2 = SelectKBest(chi2, k=k)

X_train_x2 = x2.fit_transform(X_train,train_label)
X_test = x2.transform(X_test)
'''

'k=100\nx2 = SelectKBest(chi2, k=k)\n\nX_train_x2 = x2.fit_transform(X_train,train_label)\nX_test = x2.transform(X_test)\nprint(X_train_x2.shape)\nprint(len(train_label))'

In [26]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import time
import time
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression 
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(["Melbourne","Sydney","Brisbane","Perth"])

X_train_x2,valid,train_label,valid_label=train_test_split(X_train,train_label,test_size=0.5)


model1=RandomForestClassifier(n_estimators=15, max_depth= None, max_features=9, min_samples_split= 9)
model2=LogisticRegression(C=10)
#model3=svm.SVC(kernel='linear',C=0.5,probability=True)
model4=MultinomialNB(fit_prior= True, alpha= 0.8)
model5=GaussianNB(var_smoothing=1e-08)
model1.fit(X_train_x2,train_label)
model2.fit(X_train_x2,train_label)
#model3.fit(X_train_x2,train_label)
model4.fit(X_train_x2,train_label)
model5.fit(X_train_x2.toarray(),train_label)


preds1=model1.predict(valid)
preds2=model2.predict(valid)
#preds3=model3.predict(valid)
preds4=model4.predict(valid)
preds5=model5.predict(valid.toarray())

test_preds1=model1.predict(X_test)
test_preds2=model2.predict(X_test)
#test_preds3=model3.predict(X_test)
test_preds4=model4.predict(X_test)
test_preds5=model5.predict(X_test.toarray())

preds1=le.transform(preds1) 
preds2=le.transform(preds2) 
#preds3=le.transform(preds3) 
preds4=le.transform(preds4) 
preds5=le.transform(preds5) 

stacked_predictions = np.column_stack((preds1,preds2,preds4,preds5))
'''print(preds2)'''
test_preds1=le.transform(test_preds1) 
test_preds2=le.transform(test_preds2) 
#test_preds3=le.transform(test_preds3) 
test_preds4=le.transform(test_preds4) 
test_preds5=le.transform(test_preds5) 

stacked_test_predictions = np.column_stack((test_preds1,test_preds2,test_preds4,test_preds5))
'''print(test_preds2)'''

meta_model =  RandomForestClassifier()
#print(stacked_predictions)
meta_model.fit(stacked_predictions,valid_label)


count = 0
count2 =0
for (i) in range(2500):
    #print(meta_model.predict(stacked_test_predictions)[i])
    if(meta_model.predict(stacked_test_predictions)[i]==test_label[i]):
        count =count+1
    if(meta_model.predict(stacked_predictions)[i]==train_label.iat[i]):
        count2=count2+1
print('stacker accuracy',count/2500,count2/2500)

/Users/zhaohaitian/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/zhaohaitian/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


<class 'scipy.sparse.csr.csr_matrix'>
model1 0.9772
model2 0.9856
model4 0.9616
model5 0.97
model1 0.0
model2 0.0
model4 0.0
model5 0.0


/Users/zhaohaitian/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


stacker accuracy 0.0 0.2452


In [29]:
import pandas as pd
df = pd.DataFrame(meta_model.predict(stacked_test_predictions))
df2=pd.DataFrame(answer_raw['id'])
df2['Class']=df
df2.to_csv('stacking.csv',index=False)